In [1]:
import numpy as np
import os, json
from LineageTracer.utils.create_dicts import create_test_configs_dict
from LineageTracer.test import output_embeddings, compute_tracklets, stitch_tracklets

### Specify the path to the evaluation images

Here, we specify the path to the generated crops obtained from the test images, in `crops_dir`.<br>
We also specify the path to the actual label masks, using `data_dir`.

In [2]:
crops_dir = 'crops'
data_dir = '../../../data'
project_name = 'Fluo-N2DH-GOWT1'
print("Evaluation test image crops shall be read from: {}".format(os.path.join(data_dir, project_name)))

Evaluation test image crops shall be read from: ../../../data/Fluo-N2DH-GOWT1


Specify path to `data_properties.json` which was generated in the first notebook `01-data.ipynb`. <br>
Also, specify the path to the model trained by you. <br>
Notice, that we use `object_size` equal to `int(data['mean_object_size']` during inference, and not `int(data['min_object_size'])`. We notice that this empirically gives better results!


In [3]:
#uncomment for the model trained by you
checkpoint_path = os.path.join('experiment', project_name+'-'+'demo', 'best_model.pth')
if os.path.isfile('data_properties.json'): 
    with open('data_properties.json') as json_file:
        data = json.load(json_file)
        object_size = int(data['mean_object_size'])
        num_tracklets = int(data['mean_num_tracklets'])
        mean_tracklet_length = int(data['mean_length_tracklet'])
        min_tracklet_length = int(data['min_length_tracklet'])
        std_object_size= np.maximum(data['std_object_size_x'], data['std_object_size_y'])
if os.path.isfile('train_properties.json'): 
    with open('train_properties.json') as json_file:
        data = json.load(json_file)
        num_offset_channels = int(data['num_offset_channels'])
        num_intensity_channels = int(data['num_intensity_channels'])
        num_latent_channels = int(data['num_latent_channels'])
        num_output_channels = int(data['num_output_channels'])
        margin = float(data['margin'])

Specify, where the output should be saved using variable `save_dir`.

In [4]:
save_dir = './inference'

In [5]:
if os.path.exists(checkpoint_path):
    print("Trained model weights found at : {}".format(checkpoint_path))
else:
    print("Trained model weights were not found at the specified location!")

Trained model weights found at : experiment/Fluo-N2DH-GOWT1-demo/best_model.pth


### Create `test_configs` dictionary from the above-specified parameters

Set `save_results` to `False` if one does not have the ground truth tracking on the test movie for evaluation. 

In [6]:
test_configs = create_test_configs_dict(crops_dir = crops_dir,
                                        data_dir = data_dir,
                                        project_name = project_name,
                                        checkpoint_path = checkpoint_path,
                                        num_fg_points = object_size,
                                        std_object_size = std_object_size,
                                        num_sampled_tracklets = num_tracklets,
                                        mean_tracklet_length = mean_tracklet_length, 
                                        min_tracklet_length = min_tracklet_length,
                                        save_dir = save_dir,
                                        save_results = True,
                                        num_offset_channels = num_offset_channels,
                                        num_intensity_channels =num_intensity_channels,
                                        num_latent_channels = num_latent_channels, 
                                        num_output_channels = num_output_channels,
                                        margin = margin
                                        )


`test_configs` dictionary successfully created with: 
 -- evaluation images accessed from ../../../data, 
 -- trained weights accessed from experiment/Fluo-N2DH-GOWT1-demo/best_model.pth, 
 -- output directory chosen as ./inference


### Generate embeddings from instance crops in test images

In the next cell, each mask crop from the test images is fed to the trained model. <br>
The corresponding embedding is saved with the name `t_id.npy`.

In [7]:
output_embeddings(test_configs)

`test` dataloader created! Accessing data from crops/test/
Number of tracklets in `test` directory is 92


100%|███████████████████████████████████████████| 92/92 [00:12<00:00,  7.51it/s]


### Generate tracklets from embeddings

In this step, we identify small tracklets where the learning criteria, going forward in time, is satisfied i.e. distance of an anchor at the time point $t$ to the distance of a positive sample at time point $t+1$, + the margin distance is less than distance of an anchor at time point $t$ to the distance of a negative sample at the time point $t+1$. <br>
After the execution of this step, if `save_results` was set to `True`, we report the `TRA` metric (between $0$ and $1$) which compares our predicted tracking to the ground truth tracking.

In [8]:
avg_distance, std_distance= compute_tracklets(test_configs)

100%|███████████████████████████████████████| 2572/2572 [00:36<00:00, 69.96it/s]


TRA measure: 0.968592


### (Optional) Generate Lineage Tree from tracklets by solving an Integer Linear Program (ILP)

In this step, we stitch tracklets produced by `compute_tracklets` function to form a lineage tree. <br>
For this purpose, we create a directed graph. Here, each tracklet is considered as one node. Edges are built between nodes (tracklets) which are contiguous in time. <br> 
This is an optional step and would lead to a slight increase in the `TRA` metric.

In [9]:
stitch_tracklets(test_configs, max_dist_embedding = avg_distance+5*std_distance)

100%|████████████████████████████████████████| 64/64 [00:00<00:00, 71127.57it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-19



100%|████████████████████████████████████████| 64/64 [00:00<00:00, 29086.08it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 192 rows, 76 columns and 117 nonzeros
Model fingerprint: 0x5585e63f
Variable types: 0 continuous, 76 integer (76 binary)
Coefficient statistics:


  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 124.0045841
Presolve removed 192 rows and 76 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 122.006 124.005 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.220062618947e+02, best bound 1.220062618947e+02, gap 0.0000%


100%|████████████████████████████████████████| 64/64 [00:00<00:00, 61230.72it/s]

Total runtime is 0.06509709358215332
Saving result to text file ...
Create res_track.txt



100%|███████████████████████████████████████████| 64/64 [00:47<00:00,  1.36it/s]


TRA measure: 0.969008
